<a href="https://colab.research.google.com/github/Dipendra7/2015lab1/blob/master/Pong__PPO_SNN_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
!nvidia-smi
!pip install "gym[atari, accept-rom-license]"

/bin/bash: line 1: nvidia-smi: command not found


In [100]:
import gym
import gym.spaces

DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
test_env = gym.make(DEFAULT_ENV_NAME)
print(test_env.action_space.n)

6


In [150]:
import gym
import math
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

from torch.utils.tensorboard import SummaryWriter
from spikingjelly.activation_based.examples.common.multiprocessing_env import SubprocVecEnv

from spikingjelly.activation_based import neuron, functional, layer


# Use CUDA
use_cuda = torch.cuda.is_available()
device   = torch.device('cuda' if use_cuda else 'cpu')

# Set Seed
seed = 1

random.seed(seed)
np.random.seed(seed)

torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True


if __name__ == '__main__':
    # Create Environments
    num_envs = 4
    env_name = 'PongNoFrameskip-v4'

    def make_env():
        def _thunk():
            env = gym.make(env_name)
            env.seed(seed)
            return env

        return _thunk

    envs = [make_env() for i in range(num_envs)]
    envs = SubprocVecEnv(envs)

    env = gym.make(env_name)
    env.seed(seed)


In [151]:
# Neural Network
class NonSpikingLIFNode(neuron.LIFNode):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def single_step_forward(self, x: torch.Tensor):
        self.v_float_to_tensor(x)

        if self.training:
            self.neuronal_charge(x)
        else:
            if self.v_reset is None:
                if self.decay_input:
                    self.v = self.neuronal_charge_decay_input_reset0(x, self.v, self.tau)
                else:
                    self.v = self.neuronal_charge_no_decay_input_reset0(x, self.v, self.tau)

            else:
                if self.decay_input:
                    self.v = self.neuronal_charge_decay_input(x, self.v, self.v_reset, self.tau)
                else:
                    self.v = self.neuronal_charge_no_decay_input(x, self.v, self.v_reset, self.tau)

In [152]:
class ActorCritic(nn.Module):
        def __init__(self, num_inputs, num_outputs, hidden_size, T=16, std=0.0):
            super(ActorCritic, self).__init__()

            self.critic = nn.Sequential(
                layer.Linear(num_inputs, hidden_size),
                neuron.IFNode(),
                layer.Linear(hidden_size, 1),
                NonSpikingLIFNode(tau=2.0)
            )

            self.actor = nn.Sequential(
                layer.Linear(num_inputs, hidden_size),
                neuron.IFNode(),
                layer.Linear(hidden_size, num_outputs),
                NonSpikingLIFNode(tau=2.0)
            )

            self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)

            self.T = T


In [153]:
# Define the forward function in the ActorCritic class
class ActorCritic(nn.Module): # Add this code block to the ActorCritic class definition
    def __init__(self, num_inputs, num_outputs, hidden_size, T=16):
        super(ActorCritic, self).__init__()

        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
            nn.Softmax(dim=1)
        )

        self.T = T

    def forward(self, x): # Add the forward function
        value = self.critic(x)
        probs = self.actor(x)
        dist  = Categorical(probs)
        return dist, value

In [154]:
def test_env(vis=False):
        state = env.reset()
        if vis: env.render()
        done = False
        total_reward = 0
        while not done:
            state = torch.FloatTensor(state).unsqueeze(0).to(device)
            dist, _ = model(state)
            functional.reset_net(model)
            next_state, reward, done, _ = env.step(torch.max(dist.sample(), 1)[1].cpu().numpy()[0])
            state = next_state
            if vis: env.render()
            total_reward += reward
        return total_reward

In [155]:
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        functional.reset_net(model)
        # Use dist.sample().cpu().numpy() directly for multiple environments
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

In [171]:
# ipython-input-155-91c6cdcd5b68

def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)

        # The following line flattens the state tensor and then reshapes it
        # to have the correct dimensions for the model
        state = state.flatten(start_dim=1).view(-1, num_inputs)

        dist, _ = model(state)
        functional.reset_net(model)
        # Use dist.sample().cpu().numpy() directly for multiple environments
        action = dist.sample().cpu().numpy()[0]
        next_state, reward, done, _ = env.step(action)
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

In [172]:
# GAE
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95): # removed extra indentation
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

In [175]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        # The original code was trying to perform operations on tensors of different sizes
        # The code below fixes this by iterating over the first dimension of the tensors
        for i in range(rewards[step].size(0)):
            delta = rewards[step][i] + gamma * values[step + 1][i] * masks[step][i] - values[step][i]
            gae = delta + gamma * tau * masks[step][i] * gae
            returns.insert(0, gae + values[step][i])
    return returns

In [176]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage):
    batch_size = states.size(0)
    # generates a random permutation of indices from 0 to batch_size-1
    ids = np.random.permutation(batch_size)
    # Divides the indices into batches of size mini_batch_size
    ids = np.split(ids[:batch_size // mini_batch_size * mini_batch_size], batch_size // mini_batch_size)
    for i in range(len(ids)):
        # The original code was causing an error as it was trying to access a second dimension in 1D tensors
        # The code below fixes this by removing the extra dimension access (the , :)
        yield states[ids[i]], actions[ids[i]], log_probs[ids[i]], returns[ids[i]], advantage[ids[i]]

In [180]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage):
    batch_size = states.size(0)
    # generates a random permutation of indices from 0 to batch_size-1
    ids = np.random.permutation(batch_size)
    # Divides the indices into batches of size mini_batch_size
    ids = np.split(ids[:batch_size // mini_batch_size * mini_batch_size], batch_size // mini_batch_size)
    for i in range(len(ids)):
        # The original code was causing an error as it was trying to access a second dimension in 1D tensors
        # The code below fixes this by removing the extra dimension access (the , :)
        # Additionally, the code now ensures that the indices are within the bounds of the returns tensor
        yield states[ids[i]], actions[ids[i]], log_probs[ids[i]], returns[ids[i] % returns.size(0)], advantage[ids[i] % advantage.size(0)]

In [181]:
 def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2):
        for _ in range(ppo_epochs):
            for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
                dist, value = model(state)
                functional.reset_net(model)
                entropy = dist.entropy().mean()
                new_log_probs = dist.log_prob(action)

                ratio = (new_log_probs - old_log_probs).exp()
                surr1 = ratio * advantage
                surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

                actor_loss  = - torch.min(surr1, surr2).mean()
                critic_loss = (return_ - value).pow(2).mean()

                loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()


In [184]:
def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2):
    for _ in range(ppo_epochs):
        for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
            dist, value = model(state)
            functional.reset_net(model)
            entropy = dist.entropy().mean()
            new_log_probs = dist.log_prob(action)

            ratio = (new_log_probs - old_log_probs).exp()
            # Ensure that advantage has the same shape as ratio before performing element-wise multiplication
            advantage = advantage.view(-1,1)
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss  = - torch.min(surr1, surr2).mean()
            critic_loss = (return_ - value).pow(2).mean()

            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [182]:
# Delete the writer object
del writer

# Now try initializing the SummaryWriter object
!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='./log')

In [ ]:
# Add this import statement at the beginning of your script
from torch.distributions import Categorical
num_inputs  = envs.observation_space.shape[0]
num_outputs = env.action_space.n

print('State Num: %d, Action Num: %d' % (num_inputs, num_outputs))

# Hyper params:
hidden_size      = 32
lr               = 1e-3
num_steps        = 128
mini_batch_size  = 256
ppo_epochs       = 30

T = 16

model = ActorCritic(num_inputs, num_outputs, hidden_size, T).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

max_steps = 10000
step_idx  = 0

state = envs.reset()

#writer = SummaryWriter(logdir='./log')

while step_idx < max_steps:

    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps):
        # The original code was flattening the entire state tensor
        # This line reshapes the state tensor to (batch_size, num_inputs)
        state = torch.FloatTensor(state).to(device).view(-1, num_inputs)
        dist, value = model(state)
        functional.reset_net(model)

        action = dist.sample()
        # Convert actions to numpy arrays for the environment
        action = action.cpu().numpy()

        next_state, reward, done, _ = envs.step(action)

        #log_prob = dist.log_prob(action.to(device))
        log_prob = dist.log_prob(torch.FloatTensor(action).to(device))
        entropy += dist.entropy().mean()

        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))

        states.append(state)
        # Convert actions to tensors before appending
        actions.append(torch.FloatTensor(action).to(device))

        state = next_state
        step_idx += 1

        if step_idx % 100 == 0:
            test_reward = test_env()
            print('Step: %d, Reward: %.2f' % (step_idx, test_reward))
            writer.add_scalar('Spiking-PPO-' + env_name + '/Reward', test_reward, step_idx)

    # The original code was flattening the entire next_state tensor
    # This line reshapes the next_state tensor to (batch_size, num_inputs)
    next_state = torch.FloatTensor(next_state).to(device).view(-1, num_inputs)
    _, next_value = model(next_state)
    functional.reset_net(model)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    actions   = torch.cat(actions)
    advantage = returns - values

    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)

print('----------------------------')
print('Complete')

writer.close()

State Num: 210, Action Num: 6
Step: 100, Reward: -21.00


In [166]:
for _ in range(num_steps):
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)
        functional.reset_net(model)

        action = dist.sample()
        # Convert action to a numpy array
        action = action.cpu().numpy()

        next_state, reward, done, _ = envs.step(action)

        # Convert action to a tensor and append to actions list
        actions.append(torch.FloatTensor(action).to(device))

        log_prob = dist.log_prob(torch.FloatTensor(action).to(device))
        entropy += dist.entropy().mean()

        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))

        states.append(state)

        state = next_state
        step_idx += 1

        if step_idx % 100 == 0:
            test_reward = test_env()
            print('Step: %d, Reward: %.2f' % (step_idx, test_reward))
            writer.add_scalar('Spiking-PPO-' + env_name + '/Reward', test_reward, step_idx)

    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    functional.reset_net(model)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    # Concatenate actions here, after the loop where actions have been appended
    actions   = torch.cat(actions)
    advantage = returns - values

    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 33)

In [167]:
while step_idx < max_steps:

    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps):
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)
        functional.reset_net(model)

        action = dist.sample()
        # Convert action to a numpy array
        action = action.cpu().numpy()

        next_state, reward, done, _ = envs.step(action)

        # Convert action to a tensor and append to actions list
        actions.append(torch.FloatTensor(action).to(device))

        log_prob = dist.log_prob(torch.FloatTensor(action).to(device))
        entropy += dist.entropy().mean()

        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))

        states.append(state)

        state = next_state
        step_idx += 1

        if step_idx % 100 == 0:
            test_reward = test_env()
            print('Step: %d, Reward: %.2f' % (step_idx, test_reward))
            writer.add_scalar('Spiking-PPO-' + env_name + '/Reward', test_reward, step_idx)

    # Make sure this line is indented at the same level as the for loop
    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    functional.reset_net(model)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    # Concatenate actions here, after the loop where actions have been appended
    actions   = torch.cat(actions)
    advantage = returns - values

    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (134400x3 and 210x32)

In [68]:
import warnings
warnings.filterwarnings('ignore')

In [69]:
# Taken from
# https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On/blob/master/Chapter06/lib/wrappers.py

import cv2
import numpy as np
import collections

class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super(FireResetEnv, self).__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        super(MaxAndSkipEnv, self).__init__(env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs


class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),
                                                old_space.high.repeat(n_steps, axis=0), dtype=dtype)

    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer


class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1],
                                old_shape[0], old_shape[1]), dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

def make_env(env_name):
    env = gym.make(env_name)
    env = MaxAndSkipEnv(env)
    env = FireResetEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4)
    return ScaledFloatFrame(env)

In [70]:
import torch
import torch.nn as nn        # Pytorch neural network package
import torch.optim as optim  # Pytorch optimization package

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [71]:
import numpy as np

In [72]:
!pip install norse

In [73]:
!pip install spikingjelly

In [74]:
!pip install tensorboardX
# Install the missing module using pip

In [75]:
!pip install norse

In [76]:
import gym
import math
import random
import numpy as np
from collections import namedtuple
from itertools import count
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from spikingjelly.activation_based import monitor, neuron, functional, layer
import os

from tensorboardX import SummaryWriter

In [82]:
class NonSpikingLIFNode(neuron.LIFNode):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def single_step_forward(self, x: torch.Tensor):
        self.v_float_to_tensor(x)

        if self.training:
            self.neuronal_charge(x)
        else:
            if self.v_reset is None:
                if self.decay_input:
                    self.v = self.neuronal_charge_decay_input_reset0(x, self.v, self.tau)
                else:
                    self.v = self.neuronal_charge_no_decay_input_reset0(x, self.v, self.tau)

            else:
                if self.decay_input:
                    self.v = self.neuronal_charge_decay_input(x, self.v, self.v_reset, self.tau)
                else:
                    self.v = self.neuronal_charge_no_decay_input(x, self.v, self.v_reset, self.tau)




In [83]:
# Spiking DQN algorithm
class DQSN(nn.Module):
    def __init__(self,input_shape, n_actions,hidden_size=128, output_size=1, hidden_shape=128, T=16):
        super().__init__()
        # Calculate the product of the input shape dimensions
        input_size = np.prod(input_shape)
        self.fc = nn.Sequential(
            layer.Linear(input_size, hidden_size), # Use the calculated input_size
            neuron.IFNode(),
            layer.Linear(hidden_shape, output_size),
            # Assuming NonSpikingLIFNode is defined elsewhere - If not, you'll need to define or replace it
            neuron.LIFNode(tau=2.0) # Replace NonSpikingLIFNode with neuron.LIFNode
        )

        self.T = T

    def forward(self, x):
        for t in range(self.T):
            self.fc(x)

        return self.fc[-1].v

In [88]:
# Spiking DQN algorithm
class DQSN(nn.Module):
    def __init__(self,input_shape, n_actions,hidden_size=32, output_size=1, T=16):
        super().__init__()
        # Calculate the product of the input shape dimensions
        input_size = np.prod(input_shape)
        self.fc = nn.Sequential(
            layer.Linear(input_size, hidden_size),
            neuron.IFNode(),
            layer.Linear(hidden_size, hidden_size),
            neuron.IFNode(),
            layer.Linear(hidden_size, hidden_size),
            neuron.IFNode(),
            layer.Linear(hidden_size, output_size),
            neuron.LIFNode(tau=2.0)
        )

        self.T = T

    def forward(self, x):
        for t in range(self.T):
            self.fc(x)

        return self.fc[-1].v

In [98]:
import torch.nn as nn

class DQSN(nn.Module): # Removed extra indent from this line
    def __init__(self, input_shape, n_actions):
        super(DQSN, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(input_shape[0], 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x):
        print(f"Input shape: {x.shape}") # Print input shape
        x = self.fc(x)
        print(f"Output shape: {x.shape}") # Print output shape
        return x

In [99]:
test_env = make_env(DEFAULT_ENV_NAME)
test_net = DQSN(test_env.observation_space.shape[0], test_env.action_space.n).to(device)
print(test_net)

TypeError: 'int' object is not subscriptable

In [90]:
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [91]:
import time
import numpy as np
import collections


MEAN_REWARD_BOUND = 19.0

gamma = 0.99
batch_size = 64
replay_size = 500000
learning_rate = 1e-6
sync_target_frames = 1000
replay_start_size = 50000

eps_start=1.0
eps_decay=.999985
eps_min=0.02

In [92]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class ExperienceReplay:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)

In [95]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    def play_step(self, net, epsilon=0.0, device="cpu"):

        done_reward = None
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward

In [96]:
env = make_env(DEFAULT_ENV_NAME)

net = DQSN(env.observation_space.shape, env.action_space.n).to(device)
target_net = DQSN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment="-" + DEFAULT_ENV_NAME)

buffer = ExperienceReplay(replay_size)
agent = Agent(env, buffer)

epsilon = eps_start

optimizer = optim.Adam(net.parameters(), lr=learning_rate)
total_rewards = []
frame_idx = 0

best_mean_reward = None

while True:
        frame_idx += 1
        epsilon = max(epsilon*eps_decay, eps_min)

        reward = agent.play_step(net, epsilon, device=device)
        if reward is not None:
            total_rewards.append(reward)

            mean_reward = np.mean(total_rewards[-100:])

            print("%d:  %d games, mean reward %.3f, (epsilon %.2f)" % (
                frame_idx, len(total_rewards), mean_reward, epsilon))


            writer.add_scalar("epsilon", epsilon, frame_idx)
            writer.add_scalar("reward_100", mean_reward, frame_idx)
            writer.add_scalar("reward", reward, frame_idx)

            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), DEFAULT_ENV_NAME + "-best.dat")
                best_mean_reward = mean_reward
                if best_mean_reward is not None:
                    print("Best mean reward updated %.3f" % (best_mean_reward))

            if mean_reward > MEAN_REWARD_BOUND:
                print("Solved in %d frames!" % frame_idx)
                break

        if len(buffer) < replay_start_size:
            continue

        batch = buffer.sample(batch_size)
        states, actions, rewards, dones, next_states = batch

        states_v = torch.tensor(states).to(device)
        next_states_v = torch.tensor(next_states).to(device)
        actions_v = torch.tensor(actions).to(device)
        rewards_v = torch.tensor(rewards).to(device)
        #done_mask = torch.ByteTensor(dones).to(device)
        done_mask = torch.BoolTensor(dones).to(device) # Change to BoolTensor instead of ByteTensor

        state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)

        next_state_values = target_net(next_states_v).max(1)[0]

        next_state_values[done_mask] = 0.0

        next_state_values = next_state_values.detach()

        expected_state_action_values = next_state_values * gamma + rewards_v

        loss_t = nn.MSELoss()(state_action_values, expected_state_action_values)

        optimizer.zero_grad()
        loss_t.backward()
        optimizer.step()

        if frame_idx % sync_target_frames == 0:
            target_net.load_state_dict(net.state_dict())



writer.close()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (336x84 and 28224x32)

In [ ]:
import gym
import time
import numpy as np

import torch

import collections

DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
FPS = 25

In [ ]:
!apt-get install -y xvfb x11-utils

!pip install pyvirtualdisplay==0.2.* \
             PyOpenGL==3.1.* \
             PyOpenGL-accelerate==3.1.*

!pip install gym[box2d]

import pyvirtualdisplay

In [ ]:
pip install moviepy

In [ ]:
env = gym.make(DEFAULT_ENV_NAME, render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, 'video', episode_trigger = lambda episode_id: True)

In [ ]:
import gym
import time
import numpy as np
import torch
import collections

DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
FPS = 25

model='PongNoFrameskip-v4-best.dat'
record_folder="video"
visualize=True

env = make_env(DEFAULT_ENV_NAME)
if record_folder:
        env = gym.wrappers.RecordVideo(env, record_folder,  episode_trigger = lambda episode_id: True)
net = DQN(env.observation_space.shape, env.action_space.n)
net.load_state_dict(torch.load(model, map_location=lambda storage, loc: storage))

state = env.reset()
total_reward = 0.0

while True:
        start_ts = time.time()
        if visualize:
            env.render()
        state_v = torch.tensor(np.array([state], copy=False))
        q_vals = net(state_v).data.numpy()[0]
        action = np.argmax(q_vals)

        state, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
        if visualize:
            delta = 1/FPS - (time.time() - start_ts)
            if delta > 0:
                time.sleep(delta)
print("Total reward: %.2f" % total_reward)